## Mask R-CNN Model

In [1]:
import cv2
import numpy as np
import colorsys
import time
from time import time as timer

cv2.__version__

'3.4.0'

In [2]:
#############
# Functions #
#############

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    np.random.shuffle(colors)
    return colors

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        '''
        Detect Only Person
        '''
        '''        
        if ids[i] != 1:
            continue
        '''
        '''
        Detect Only Person
        '''        
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        #image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255,0), 5)
        #image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image

In [3]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Path to Shapes trained weights
#SHAPES_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_shapes.h5")

/home/cfchen/anaconda3/envs/MaskRCNN/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Configurations

In [4]:
# Run one of the code blocks
# MS COCO Dataset

sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
config = coco.CocoConfig()

COCO_DIR = "/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/COCO2017/2014"  # TODO: enter value here

In [5]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Notebook Preferences

In [6]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [7]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [8]:
# Build validation dataset
if config.NAME == 'shapes':
    dataset = shapes.ShapesDataset()
    dataset.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
elif config.NAME == "coco":
    dataset = coco.CocoDataset()
    #dataset.load_coco(COCO_DIR, "minival")
    dataset.load_coco(COCO_DIR, "val")


# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

loading annotations into memory...
Done (t=3.46s)
creating index...
index created!
Images: 40137
Classes: ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


## Load Model

In [9]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Videos

In [10]:
def make_480p():
    capture.set(3, 640)
    capture.set(4, 360)
    
def change_res(width, height):
    capture.set(3, width)
    capture.set(4, height)

In [11]:
make_480p()
change_res(640, 360)

NameError: name 'capture' is not defined

In [13]:
def rescale_frame(frame, percent=75):
    width  = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [89]:
#capture = cv2.VideoCapture(0)
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 30.0, (1920,1080))

In [90]:
print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

Video Dim: 1920.0  X  1080.0


In [91]:
stime = time.time()
start = timer()

count = 0;
while(True):
    ret, frame = capture.read() 
    
    if ret == True:
        frame = cv2.resize(frame, (640, 360)) 
        
        #frame = rescale_frame(frame, percent=50)
        
        print("Video Dim:  {}".format(frame.shape))

        results = model.detect([frame], verbose=0)
        #
    
        r = results[0]
    
        frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'])

        out.write(frame)

        cv2.imshow('frame', frame)

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

    else:
        break
        
print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nElapsed time = ' + str(timer() - start) + ' s\n')


Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  (360, 640, 3)
Video Dim:  

## Pictures

In [28]:
        count = count + 1

print("Video Dim:  {}".format(frame.shape))
frame = rescale_frame(frame, percent=50)
frame = cv2.resize(frame, (640, 360)) 
resize= cv2.resize(frame, (640, 360)) 

++++++++ Video End ++++++++

Elapsed time = 75.03091025352478 s



In [20]:
stime = time.time()
start = timer()

count = 0;
while(True):
    ret, frame = capture.read() 

    if ret == False:
        break;
    else:
        count = count + 1
    
    #
    '''
    height , width , layers =  frame.shape
    height = float(height)
    width = float(width)
    print(type(height))
    new_h = height / 2
    new_w = width / 2
    frame = cv2.resize(frame, (new_w, new_h)) 
    '''
    
    frame = cv2.resize(frame, (640, 360)) 
    #
    results = model.detect([frame], verbose=0)
    #
    r = results[0]
    
    frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'])
      
        
    out.write(frame)
    cv2.imshow('frame', frame)
    if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
        break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nElapsed time = ' + str(timer() - start) + ' s\n')

++++++++ Video End ++++++++

Elapsed time = 0.0014584064483642578 s

